This is the second part of the final exam for AcF 351b: Python Stream.   

Students are expected to act according to the highest ethical standards. All students enrolled at Lancaster University are to perform their academic work according to standards set by faculty members, departments, schools and colleges of the university; and cheating and plagiarism constitute fraudulent misrepresentation for which appropriate sanctions are warranted and will be applied. Please note that any form of violation of the following rules will be treated as plagiarism

1. Answer the questions yourself without asking others for assistance. This is a test of your ability of data science and computer programming. 
2. Do not share the questions or your answers with anyone. This includes posting the questions or your solutions publicly on services like quora, stackoverflow, or github.

**We will run a system to detect any kind of plagiarism, e.g., coding scripts with high similarities.**



**Do NOT erase the `#export` at the top of any cells as it is used by `notebook2script.py` to extract cells for submission.**

## Do NOT change the following cell!

In [47]:
#export
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd, QuarterEnd, QuarterBegin
from dateutil import parser
import time
import matplotlib.pyplot as plt

try:
    import wrds
except:
    !pip3 install wrds

## If you need extra modules, use the following cell to import them.

In [48]:
#export
# imported extra moduels:
import wrds

In this notebook, you are going to analyze the data of U.S. and U.K corporate bonds in the U.S. market. For corporate bonds, the typical variable of interest is [bond yield](https://www.investopedia.com/terms/b/bond-yield.asp). We are not going to elaborate the definition of bond yield and how bond yield is calculated. For our purpose, we only need to know that bond yields are **inversely** related to bond prices. If investors prefer holding some bonds, the price of the bonds increases and, consequently, the yield decreses.   

_acf351b_bonds.csv_ provides historical yields and trading quantities for a sample of U.S. and U.K. bonds in the U.S. market between Jan 2015 and Dec 2017.    

The following table shows the data dictionary.

| Column Name | Description                                                    | Expected Values |   
|-------------|----------------------------------------------------------------|-----------------|
| issue_id    | A unique identifier for each bond issue                        | Number          |
| date        | The date on which the bond transaction occurs                  | Text            |
| yield       | The calculated bond yields for the transaction                 | Number          |
| month       | The end date of the month in which the bond transaction occurs | Number          |
| quant       | The dollar value of the transaction                        | Number          |
| offering_date       | The date on which the bond is issued | Text          |
| maturity       | The date on which the bond matures | Text          |
| country_GBR       | Whether the bond issuer is domiciled in the UK. Yes==1, No==0 | Number          |
| permno       | A unique identifier to link bond issuers to their stock returns | Number          |   


The following code script reads the data into the memory, and passes it to a dataframe named _df_. DO NOT CHANGE IT.    
However, you can add extra code cells below it to have an overview.


In [49]:
#export
#df = pd.read_csv("https://frankxu1987.weebly.com/uploads/6/2/5/8/62583677/acf351b_bonds.csv")
df = pd.read_csv("acf351b_bonds.csv")
df['date']=pd.to_datetime(df['date'], format="%Y-%m-%d")
df.head(1)

issue_id       date  yield       month     quant offering_date    maturity  \
0       5.0 2015-01-27    NaN  2015-01-31  760000.0    1993-05-20  2023-05-15   

   country_GBR   permno  
0            0  92355.0

**We are going to make use of the data on WRDS. The following code script establishes the connection to WRDS database.**

In [50]:
#export
conn = wrds.Connection()

# liy78 
# Natasha370lyx.

Enter your WRDS username [mengxiangpeng]:liy78
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
Loading library list...
Done


## Question 1: Using SQL to retrieve data of stock returns and compute 90-day return volatilities  

CRSP is a dataset of historical stock prices (variable: prc) and stock returns (variable: ret) for most stocks listed in the U.S. stock exchanges. Each stock has a unique identifier called _permno_. In this question, we are going to write PostgreSQL scripts to retrieve stock returns for all bond issuers in our bond data _df_.

**Question 1.1: write a piece of SQL scripts to retrieve stock return (variable: ret) and stock trading date (date) from CRSP (dataset: crsp.dsf) between July 1st, 2014 and Dec 31st, 2017. (10 pts)**     
**A part of the script is provided. Complete the SQL query based on the information in Question 1.1**

In [55]:
#export
# Code script for Q 1.1
# Complete the code script below.

stmt="""

select permno, date
from crsp.dsf
where permno in {}

and 
date between '2014-07-01' AND '2017-12-31'

""".format(tuple(df.loc[df['permno'].notnull(), 'permno'].unique()))


df_vol = conn.raw_sql(stmt, date_cols=['date'])

In [ ]:
#export
# the length of the dataframe should be close to 68000
print(len(df_vol))

**Question 1.2: Compute the past 90-day return volatility. For each stock on each trading day, compute the 90-day volatility (standard deviation) of returns, and generate a new column of _df_vol_ titled `vol`. Note that the 90 days include the trading day being considered. (5 pts)**

**For instance, for the stock with a permno 93013 on Dec 28, 2017, the 90-day volatility is the standard deviation of its returns of the most recent 90 days inclusive of Dec 28, 2017.**  

_hint: use groupby() and rolling() function. The answer probably would not take more than 3 lines._

In [ ]:
#export
# Code script for Q 1.2
# write your code script below.

stmt="""

select variance(permno)
from (select date_trunc('date', timestamp) as mon, sum(permno) as sum_vals
      from t
      group by date
      and 
      date between '2014-07-01' AND '2017-12-31'
     ) t;
"""

df_vol = conn.raw_sql(stmt, date_cols=['date'])

**Question 1.3: Left merge dataframes df and df_vol based on permno and date, and name the new dataframe with _df_. Only Keep the variable `vol` from df_vol. In other words, the variable `ret` from _df_vol_ should not appear in the final dataframe. (5 pts)**

In [ ]:
#export
# Code script for Q 1.3
# Write your code script below 




## Question 2: Bond Ratings
    
Some bonds have higher credit ratings, meaning that investors are confident that the bond issuers are able to pay off the bonds when they are due. Some bonds have lower credit ratings, and investors may demand higher risk compensations, resulting in higher bond yields. In this question, we are going to find out the credit ratings for each bond on each date in our dataframe _df_.

_acf351b_ratings.csv_ provides historical credit ratings for all bond issues in the spreadsheet.

In [ ]:
#export
df_rating=pd.read_csv("https://frankxu1987.weebly.com/uploads/6/2/5/8/62583677/acf351b_ratings.csv")
df_rating.head(1)

The variable _issue_id_ is the identifier of bonds. 
The variable _rating_type_ indicates the rating agencies. _MR_ denotes Moody's Rating, _SPR_ indicates Standard and Poor's Rating, and _FR_ indicates Fitch's Rating. 
The variable _rating_date_ is the date on which the rating agencies assign their ratings.
The variable _rating_ is the bond rating assigned by the rating agency.  


Each rating agency has their own rating scales, but they are comparable as follows:

| MR                     | SPR                  | FR                     | Standard Rating |
| ---------------------- | -------------------- | ---------------------- | --------------- |
| Aaa                    | AAA                  | AAA                    | AAA             |
| Aa1, Aa2, Aa3          | AA+, AA, AA-         | AA+, AA, AA-           | AA              |
| A1, A2, A3             | A+, A, A-            | A+, A, A-              | A               |
| Baa1, Baa2, Baa3       | BBB+,BBB, BBB-       | BBB+,BBB, BBB-         | BBB             |
| Ba1, Ba2, Ba3          | BB+,BB,BB-           | BB+,BB,BB-             | BB              |
| B1, B2, B3             | B+,B,B-              | B+,B,B-                | B               |
| Caa1, Caa2, Caa3, Ca,C | CCC+,CCC,CCC-,CC,C,D | CCC, CC, C, DDD, DD, D | C               |  
| NR | NR | NR | NR               |  

with AAA being the highest credit rating, AA being one tier lower, and so forth. NR indicates `Not Rated`

**Question 2.1: Convert all ratings to standard ratings based on the table above, and generate a new column of _df_rating_ named `std_rating`. (5 pts)**

In [ ]:
#export
# Code script for Q 2.1
# Write your code script below 



**Question 2.2: Using _df_rating_, locate the latest _standard ratings_ prior to the trade dates (variable `date`) for each bond issue in _df_, and generate a new variable named `rating` in _df_. (10 pts)**

**For example, the bond with an issue id of 647968 was traded on March 31st, 2016, then for this transaction you should try to locate the latest credit rating of the bond prior to March 31st, 2016. If multiple credit rating agencies (S&P, Moody’s and Fitch) rate the bond issue on the same dates , select the LOWEST one.**

_Hint: Sort the dataframes **df** and **df_rating** appropriately. Then use the function [pd.merge_asof()](https://pandas.pydata.org/pandas-docs/version/0.25.0/reference/api/pandas.merge_asof.html). The answer probably would not take more than 5 lines._

In [ ]:
#export
#Code script for Q 2.2
#Write your code script below



## Question 3: Treasury Yield and Yield Spreads

Bond yields are also affected by macroeconomic fundamentals, esp. U.S. treasury yields. U.S. treasuries are typically assumed to be risk-free, and, thus, the differences between the corporate bond yields and U.S. treasury yields should reflect the credit risk of the bond issuers. In this question, we are going to derive bond yield spreads from bond yields variable `yield`.  

_zero_coupon_yld.csv_ provides historical U.S. treasury yields.

In [ ]:
#export
df_treasury = pd.read_csv("https://frankxu1987.weebly.com/uploads/6/2/5/8/62583677/zero_coupon_yld.csv")
df_treasury.head(1)

The variables _SVENYXX_ refer to the U.S. treasury par yields where XX indicates the maturity. For instance, SVENY05 refers the 5-year U.S. treasury par yields.

**Question 3.1: Left merge df and df_treasury on `date`. (5 pts)**    
_Hint: make sure that the `date` variables are datetime formats in both dataframes._

In [ ]:
#export
#Code script for Q 3.1
#Write your code script below



**Question 3.2: Compute the difference in years between maturity and date in df, and generate a variable named ttm to represent time to maturity. (5 pts)**   

**For instance, if a bond matures on March 31st, 2025 and the trading date is March 15th, 2016, then the ttm (Time To Maturity) for the bond is 9.04 years.**

In [ ]:
#export
#Code script for Q 3.2
#Write your code script below



**Question 3.3: For each row in _df_, i.e., for each bond on each trade date in _df_, locate the U.S. treasuries with maturities closest to the bond issues’ ttm(s), and then generate a variable named treasury_yield to represent the treasury yields of the closest maturities. (10 pts)**  

**For example, if a bond is traded on March 15th, 2016 and its time to maturity (ttm) is 9.04 years, then you should locate the par yield of US 9-year (the closest time to maturity) treasury on March 15th, 2016 as the treasury_yield of the bond.**

In [ ]:
#export
#Code script for Q 3.3
#Write your code script below




Finally, we can compute the bond yield spreads `spread`

In [ ]:
#export
df['spread']= df['yield']-df['treasury_yield'];

## Q4: Winsorization and Outliers

Raw data consists of outliers most of the time, if not all. An [outlier](https://en.wikipedia.org/wiki/Outlier) is a data point that differs significantly from other observations. A typical way to deal with outliers is to drop them from analysis. In this section, we will conduct simple winsorization to eliminate potential outliers.

**Q4.1 Compute the 1st percentile and 99th percentile values for variables `spread` and `ttm`, and PRINT out the final values with _print()_ function. (5 pts)**

In [ ]:
#export
#Code script for Q 4.1
#Write your code script below



**Q4.2 Drop all observations (rows) of dataframe _df_ of which the values of `spread` or `ttm` are not in the range of corresonding 1st percentile and 99th percentile. (5 pts)**

In [ ]:
#export
#Code script for Q 4.2
#Write your code script below



## Congratulations for finishing the second part of the final exam !
### Remeber to save your Jupyter Notebook.
### Now it's a good time to submit for grading.

Please uncomment and run the cell below. Your code will be generated in the folder named `second_part`, please upload `submission.py` file AND the Jupyter Notebook (`AcF_351b_Python_Stream_Final_Exam_Part_II.ipynb`) together.

In [ ]:
#%run helpers/notebook2script2 second_part

Now that you have a complete database of US and British bond transactions from January 2015 to December 2017, you can start to explore and analyze the data. You should try to answer the following questions

**1. Using a figure to illustrate in which month the difference in yield spreads of US and UK bonds start to increase ? (5 pts)**

**2. Using a figure to illustrate in which month the difference in 90-day volatilities of US and UK bond issuers start to increase ? (5 pts)**

**3. What do the bond transaction data demonstrate about the finanial markets? (5 pts)**

Finish your answers in the `acf351b_python.docx`